In [1]:
import checklist
import spacy
import itertools
from checklist.model_api import circle_model
import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
import numpy as np
import spacy
from checklist.perturb import Perturb
from checklist.pred_wrapper import PredictorWrapper


08/13/2020 21:23:36 - INFO - transformers.file_utils -   PyTorch version 1.3.0+cu100 available.
08/13/2020 21:23:39 - INFO - transformers.file_utils -   TensorFlow version 2.0.0 available.


In [2]:
test_model = circle_model()
wrapped_pp = PredictorWrapper.wrap_softmax(test_model.eval)
editor = checklist.editor.Editor(language='chinese')


In [3]:
suite = TestSuite()

测试分类模型对POS+Vocabulary的能力，构造MFT INV DIR三种测试方案
首先构造MFT测试
我们首先利用词库提供一些地名，构造一些简单样本进行分类准确率的测试

In [4]:
ret = editor.template('去{company}是地铁几号线,我该怎么去', labels=468, nsamples=200)
ret += editor.template('坐什么公交可以到{company}', labels=467, nsamples=200)
print(ret.data[:5])
test = MFT(ret.data, labels=ret.labels, name='vocabluary',
           capability='vocab', description='test model vocab ')
suite.add(test)
# test.run(wrapped_pp)


['去蚂蚁金服是地铁几号线,我该怎么去', '去腾讯是地铁几号线,我该怎么去', '去特斯联是地铁几号线,我该怎么去', '去海康威视是地铁几号线,我该怎么去', '去小马智行是地铁几号线,我该怎么去']


构造INV测试, 使用模型生成一些动词，利用这些动词对样本随机替换
模型应该保持预测结果不变

In [5]:
def change_v(data):
    if data.find('去') > -1:
        new_data=data.replace('去', random.choice(v))
    else:
        new_data=data.replace('坐', random.choice(v2))
    return [new_data]


In [6]:
# 这里复用mft单元生成的数据，由于INV测试只比较模型是否对扰动前后样本预测结果相同，因此INV数据不需要label标签
data = ret.data
print(data[:3])
v = editor.suggest('去{company}是地铁几号线？我应该怎么{mask}呢？')[:20]
v2 = editor.suggest('我应该{mask}什么公交可以到{company}')[:20]
t = Perturb.perturb(data, change_v)
inv = INV(**t, name='verb test', capability='vocab')
suite.add(inv)
# 将测试数据重新写入文件中
# examples = inv.to_raw_file('/work/QA_task/roberta-1.1/BERTCN/bertcn-pytorch-r1.1/data/checklist/dev_468_inv.txt')

['去蚂蚁金服是地铁几号线,我该怎么去', '去腾讯是地铁几号线,我该怎么去', '去特斯联是地铁几号线,我该怎么去']
08/13/2020 21:25:29 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
08/13/2020 21:25:30 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.f12a4f986e43d8b328f5b067a641064d67b91597567a06c7b122d1ca7dfd9741
08/13/2020 21:25:30 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidd

最后生成DIR测试，我们希望肯定转为否定能降低模型对类别判断的置信度(概率)
我们首先需要编写一个自定义的期望函数，即认为概率下降/上升多少该case为pass/fail


In [7]:
# 我们期望采取否定语法后，所有样本不改变预测结果
def high_confidence(x, pred, conf, label=None, meta=None):
    return conf.max() < 0.6
expect_fn = Expect.single(high_confidence)

In [8]:
# 继续沿用INV的data
t = Perturb.perturb(data, Perturb.add_negation)
print(t.data[:3])
dir = DIR(**t, expect=expect_fn, name="negation test", capability='vocab')
suite.add(dir)
# 将测试数据重新写入文件中
# examples = inv.to_raw_file('/work/QA_task/roberta-1.1/BERTCN/bertcn-pytorch-r1.1/data/checklist/dev_468_dir.txt')

[['去蚂蚁金服是地铁几号线,我该怎么去', '去蚂蚁金服别是地铁几号线,我该怎么去'], ['去腾讯是地铁几号线,我该怎么去', '去腾讯并不是地铁几号线,我该怎么去'], ['去特斯联是地铁几号线,我该怎么去', '去特斯联非地铁几号线,我该怎么去']]


In [9]:
suite.run(wrapped_pp)


08/13/2020 21:25:39 - INFO - checklist.model_api -   ***** Running devaluation *****
08/13/2020 21:25:39 - INFO - checklist.model_api -     Num examples = 400
08/13/2020 21:25:39 - INFO - root -   Start dev!
Iteration:   1%|          | 4/400 [00:00<00:10, 37.90it/s]Running vocabluary
Predicting 400 examples
Iteration: 100%|██████████| 400/400 [00:09<00:00, 41.38it/s]
08/13/2020 21:25:48 - INFO - root -   Start dev!
Iteration: 100%|██████████| 400/400 [00:09<00:00, 43.17it/s]
08/13/2020 21:25:58 - INFO - checklist.model_api -   ***** Running devaluation *****
08/13/2020 21:25:58 - INFO - checklist.model_api -     Num examples = 800
08/13/2020 21:25:58 - INFO - root -   Start dev!
Iteration:   0%|          | 0/800 [00:00<?, ?it/s]本轮eval耗时共18.99524760246277
Running verb test
Predicting 800 examples
Iteration: 100%|██████████| 800/800 [00:19<00:00, 41.99it/s]
08/13/2020 21:26:17 - INFO - root -   Start dev!
Iteration: 100%|██████████| 800/800 [00:19<00:00, 41.24it/s]
08/13/2020 21:26:36 - 

In [10]:
suite.summary()

vocab

vocabluary
Test cases:      400
Fails (rate):    5 (1.2%)

Example fails:
383 (0.5) 坐什么公交可以到大众点评
----
383 (0.5) 坐什么公交可以到大众点评
----
383 (0.5) 坐什么公交可以到大众点评
----


verb test
Test cases:      400
Fails (rate):    48 (12.0%)

Example fails:
468 (0.5) 去蚂蚁金服是地铁几号线,我该怎么去
194 (0.3) 做蚂蚁金服是地铁几号线,我该怎么做

----
467 (0.4) 坐什么公交可以到第四范式
41 (0.3) 问什么公交可以到第四范式

----
467 (0.6) 坐什么公交可以到头条
121 (0.4) 换什么公交可以到头条

----


negation test
Test cases:      400
Fails (rate):    50 (12.5%)

Example fails:
467 (0.5) 坐什么公交可以到影谱科技
467 (0.6) 坐什么公交别到影谱科技

----
467 (0.6) 坐什么公交可以到旷视科技
467 (0.7) 坐什么公交别到旷视科技

----
467 (0.6) 坐什么公交可以到中科曙光
467 (0.7) 坐什么公交无法到中科曙光

----






In [11]:
a, b = suite.visual_summary_table()

Please wait as we prepare the table data...


In [13]:
from checklist.viewer.suite_summarizer import SuiteSummarizer
SuiteSummarizer(a, b)

SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'vocabluary', 'descri…